In [2]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import gc

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

#取消科學記號
pd.options.display.float_format = '{:.7f}'.format  #pandas 取消科學記號，只顯示小數點第二位

In [3]:
app_train = pd.read_csv('C:/Shared/machine_learning/home-credit-default-risk/application_train.csv')
app_test = pd.read_csv('C:/Shared/machine_learning/home-credit-default-risk/application_test.csv')
final_train = pd.read_csv('C:/Shared/machine_learning/home-credit-default-risk/final_train.csv')
final_test = pd.read_csv('C:/Shared/machine_learning/home-credit-default-risk/final_test.csv')

In [4]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import gc

def model(features, test_features, encoding = 'ohe', n_folds = 5):
    
    """Train and test a light gradient boosting model using
    cross validation. 
    
    Parameters
    --------
        features (pd.DataFrame): 
            dataframe of training features to use 
            for training a model. Must include the TARGET column.
        test_features (pd.DataFrame): 
            dataframe of testing features to use
            for making predictions with the model. 
        encoding (str, default = 'ohe'): 
            method for encoding categorical variables. Either 'ohe' for one-hot encoding or 'le' for integer label encoding
            n_folds (int, default = 5): number of folds to use for cross validation
        
    Return
    --------
        submission (pd.DataFrame): 
            dataframe with `SK_ID_CURR` and `TARGET` probabilities
            predicted by the model.
        feature_importances (pd.DataFrame): 
            dataframe with the feature importances from the model.
        valid_metrics (pd.DataFrame): 
            dataframe with training and validation metrics (ROC AUC) for each fold and overall.
        
    """
    
    #ID 取出
    # Extract the ids
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']
    
    #目標取出
    # Extract the labels for training
    labels = features['TARGET']
    
    #刪除train 的目標和ID ，刪除test的ID
    # Remove the ids and target
    features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    
    #分兩種方法'ohe':將train和test都做one hot建立虛擬變數(攤平)，並且根據兩者都有得col項目合併
    # One Hot Encoding
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)
        
        # Align the dataframes by the columns
        features, test_features = features.align(test_features, join = 'inner', axis = 1)
        
        # No categorical indices to record
        cat_indices = 'auto'
    
    
    
    # Integer label encoding
    elif encoding == 'le':
        
        # Create a label encoder
        label_encoder = LabelEncoder()
        
        # List for storing categorical indices
        cat_indices = []
        
        #enumerate() 將資料變成((0,apptrain.columns[0]), (1,apptrain_columns[1])....)一對一放進 i 和 col跑迴圈
        #LabelEncoder.fit_transform 將 object變數做重新編碼從 0 開始
        #試試這行 : label_encoder.fit_transform(['a','a','b','c','a','b'])
        #cat_indices 為object的索引
        # Iterate through each column
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                # Map the categorical features to integers
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

                # Record the categorical indices
                cat_indices.append(i)
    
    # Catch error if label encoding scheme is not valid
    #encoding參數給錯就報錯
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    #取出特徵名稱
    feature_names = list(features.columns)
    
    #dataframe 轉成np.array
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 50)
    
    # Empty array for feature importances
    #建立長度和特徵名稱一樣長且全為 0 的 np.array
    feature_importance_values = np.zeros(len(feature_names))
    
    
    # Empty array for test predictions
    #建立長度和測試資料筆數一樣長且全為 0 的 np.array
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    #建立長度和訓練資料筆數一樣長且全為 0 的 np.array
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    #記錄驗證和訓練的成績
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    # 迭代所有交叉驗證集
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        #交叉訓練資料
        train_features, train_labels = features[train_indices], labels[train_indices]
        
        # Validation data for the fold
        #交叉驗證資料
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        #建立 LGBM模型參數
        model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                                   class_weight = 'balanced', learning_rate = 0.05, 
                                   reg_alpha = 0.1, reg_lambda = 0.1, 
                                   subsample = 0.8, n_jobs = -1, random_state = 50)
        #訓練開始
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics

In [5]:
model(final_train,final_test)

Training Data Shape:  (307507, 869)
Testing Data Shape:  (48744, 869)
Training until validation scores don't improve for 100 rounds.
[200]	valid's binary_logloss: 0.529605	valid's auc: 0.781281	train's binary_logloss: 0.515641	train's auc: 0.834975
[400]	valid's binary_logloss: 0.502599	valid's auc: 0.782659	train's binary_logloss: 0.47567	train's auc: 0.872659
Early stopping, best iteration is:
[372]	valid's binary_logloss: 0.505889	valid's auc: 0.782824	train's binary_logloss: 0.480648	train's auc: 0.868228
Training until validation scores don't improve for 100 rounds.
[200]	valid's binary_logloss: 0.529414	valid's auc: 0.789708	train's binary_logloss: 0.517895	train's auc: 0.832897
[400]	valid's binary_logloss: 0.501943	valid's auc: 0.791013	train's binary_logloss: 0.477773	train's auc: 0.871454
Early stopping, best iteration is:
[397]	valid's binary_logloss: 0.502322	valid's auc: 0.791045	train's binary_logloss: 0.478337	train's auc: 0.870917
Training until validation scores don't 

(       SK_ID_CURR    TARGET
 0          100001 0.3216596
 1          100005 0.5739773
 2          100013 0.2117655
 3          100028 0.2800243
 4          100038 0.6988437
 ...           ...       ...
 48739      456221 0.3580784
 48740      456222 0.4044381
 48741      456223 0.0558504
 48742      456224 0.2500496
 48743      456250 0.7262148
 
 [48744 rows x 2 columns],                               feature  importance
 0                          Unnamed: 0  37.2000000
 1                               index   0.0000000
 2                         AMT_ANNUITY 138.6000000
 3                          AMT_CREDIT  62.6000000
 4                     AMT_GOODS_PRICE  74.6000000
 ..                                ...         ...
 864   CC_NAME_CONTRACT_STATUS_nan_MAX   0.0000000
 865  CC_NAME_CONTRACT_STATUS_nan_MEAN   0.0000000
 866   CC_NAME_CONTRACT_STATUS_nan_SUM   0.0000000
 867   CC_NAME_CONTRACT_STATUS_nan_VAR   0.0000000
 868                          CC_COUNT   2.4000000
 
 [869 rows